In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import autokeras as ak

In [ ]:
directory = 'blue/sea_turtle_images'

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

train_data = ak.image_dataset_from_directory(
    directory,
    # Use 20% data as testing data.
    validation_split=0.2,
    subset="training",
    # Set seed to ensure the same split when loading testing data.
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

test_data = ak.image_dataset_from_directory(
    directory,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(train_data, epochs=20)


In [ ]:
# Predict with the best model.
predicted_y = clf.predict(test_data)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(test_data))

In [ ]:
# Customized Search Space
# For advanced users, you may customize your search space by using AutoModel
# instead of ImageClassifier. You can configure the ImageBlock for some
# high-level configurations, e.g., block_type for the type of neural network to
# search, normalize for whether to do data normalization, augment for whether to
# do data augmentation. You can also do not specify these arguments, which would
# leave the different choices to be tuned automatically. See the following
# example for detail.

In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(train_data, epochs=20)


In [ ]:
# The usage of AutoModel is similar to the functional API of Keras. Basically, you are
# building a graph, whose edges are blocks and the nodes are intermediate outputs of
# blocks. To add an edge from input_node to output_node with output_node =
# ak.[some_block]([block_args])(input_node).

# You can even also use more fine grained blocks to customize the search space even
# further. See the following example.


In [ ]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(train_data, epochs=20)
